# 分析电影数据

如果DataFrame中的某行同属于多个分类，则事情就会有点复杂。看一下MovieLens 1M数据集，14章会更深入地研究它：

In [5]:
import pandas as pd
import numpy as np


In [6]:
mnames = ['movie_id', 'title', 'genres']


movies = pd.read_table('../../datasets/movielens/movies.dat', 
                       engine='python',
                       sep='::',
                       header=None, names=mnames)

movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
all_genres = []


# 抽取类型
for x in movies.genres:
    all_genres.extend(x.split('|'))
    

genres = pd.unique(all_genres)


# 全部电影类型
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

构建指标DataFrame的方法之一是从一个全零DataFrame开始：

In [8]:
zero_matrix = np.zeros((len(movies), len(genres)))

zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


现在，迭代每一部电影，并将dummies各行的条目设为1。要这么做，我们使用dummies.columns来计算每个类型的列索引：

In [10]:
gen = movies.genres[0]

print(gen)

gen.split('|')

Animation|Children's|Comedy


['Animation', "Children's", 'Comedy']

In [11]:
movie_types = dummies.columns

movie_types

Index(['Animation', 'Children's', 'Comedy', 'Adventure', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Sci-Fi',
       'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir', 'Western'],
      dtype='object')

In [12]:
movie_types.get_indexer(gen.split('|'))

array([0, 1, 2])

然后，根据索引，使用.iloc设定值：

In [13]:
for i, gen in enumerate(movies.genres):
    
    now_tags = gen.split('|')
    indices = dummies.columns.get_indexer(now_tags)
    print("for循环index: %s, \t indices: %s, \t 电影类型: %s" % (i, indices, now_tags))
    dummies.iloc[i, indices] = 1


for循环index: 0, 	 indices: [0 1 2], 	 电影类型: ['Animation', "Children's", 'Comedy']
for循环index: 1, 	 indices: [3 1 4], 	 电影类型: ['Adventure', "Children's", 'Fantasy']
for循环index: 2, 	 indices: [2 5], 	 电影类型: ['Comedy', 'Romance']
for循环index: 3, 	 indices: [2 6], 	 电影类型: ['Comedy', 'Drama']
for循环index: 4, 	 indices: [2], 	 电影类型: ['Comedy']
for循环index: 5, 	 indices: [7 8 9], 	 电影类型: ['Action', 'Crime', 'Thriller']
for循环index: 6, 	 indices: [2 5], 	 电影类型: ['Comedy', 'Romance']
for循环index: 7, 	 indices: [3 1], 	 电影类型: ['Adventure', "Children's"]
for循环index: 8, 	 indices: [7], 	 电影类型: ['Action']
for循环index: 9, 	 indices: [7 3 9], 	 电影类型: ['Action', 'Adventure', 'Thriller']
for循环index: 10, 	 indices: [2 6 5], 	 电影类型: ['Comedy', 'Drama', 'Romance']
for循环index: 11, 	 indices: [ 2 10], 	 电影类型: ['Comedy', 'Horror']
for循环index: 12, 	 indices: [0 1], 	 电影类型: ['Animation', "Children's"]
for循环index: 13, 	 indices: [6], 	 电影类型: ['Drama']
for循环index: 14, 	 indices: [7 3 5], 	 电影类型: ['Action', 'Adventure',

In [14]:
dummies.iloc[0, [0, 1, 2]]

Animation     1.0
Children's    1.0
Comedy        1.0
Name: 0, dtype: float64

In [15]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


然后，和以前一样，再将其与movies合并起来：

In [16]:
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [19]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))


movies_windic.iloc[: , 0: 10]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3879,3949,Requiem for a Dream (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3880,3950,Tigerland (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3881,3951,Two Family House (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0


> 笔记：对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入NumPy数组，然后结果包装在DataFrame中。

## 案例二 随机数 -> 矩阵

一个对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数：

In [40]:
np.random.seed(12345)

然后，和以前一样，再将其与movies合并起来：

生成 0 ~ 1 的随机样本数

In [41]:
values = np.random.rand(10)

values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [44]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

res_cut = pd.cut(values, bins)

res_cut

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64, right]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [45]:
pd.value_counts(res_cut)

(0.6, 0.8]    3
(0.2, 0.4]    2
(0.4, 0.6]    2
(0.8, 1.0]    2
(0.0, 0.2]    1
dtype: int64

我们用numpy.random.seed，使这个例子具有确定性。本书后面会介绍pandas.get_dummies。

In [46]:
# 意义: 随机数据, 在 面元划分区间

pd.get_dummies(res_cut)

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
